In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import sys
sys.path.append("../src")
from data_proc import *
from baselines import *
from sklearn.linear_model import LinearRegression, LogisticRegression

pd.options.mode.chained_assignment = None

# Data loading & preprocessing

In [2]:
# Load in raw data file
data = pd.read_csv("../data/warfarin.csv")

# Preprocess data
data = preprocess(data)

print("Number of records: {}".format(len(data)))
data.head()

Number of records: 5528


,daily-dosage,dosage-level,Age,Height (cm),Weight (kg),Asian,African-American,Race-Unknown,Enzyme,Amiodarone,...,CYP2C9-22,CYP2C9-23,CYP2C9-33,CYP2C9-Unknown,warfarin-treatment-3,warfarin-treatment-4,Current Smoker-1,Current Smoker-0,Congestive Heart Failure and/or Cardiomyopathy-1,Congestive Heart Failure and/or Cardiomyopathy-0
0,7.000000,high,6.0,193.04,115.7,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,6.000000,medium,5.0,176.53,144.2,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,7.571429,high,4.0,162.56,77.1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4.000000,medium,6.0,182.24,90.7,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,6.000000,medium,5.0,167.64,72.6,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


# Evaluate performance of baseline models

In [3]:
def accuracy(s1, s2):
    return (s1 == s2).mean()

In [4]:
print("Evaluate performance (fraction of right decisions) of baseline models...")

# Baseline 1: Fixed-dose
print("Fixed-dose: {}".format(accuracy(data['dosage-level'], 'medium')))

# Baseline 2: Clinical Dosing Algorithm
print("Clinical Algorithm: {}".format(accuracy(data['dosage-level'], clinical_predict(data))))

# Baseline 3: Pharmacogenetic Dosing Algorithm
print("Pharmacogenetic Algorithm: {}".format(accuracy(data['dosage-level'], genetic_predict(data))))

Evaluate performance (fraction of right decisions) of baseline models...
Fixed-dose: 0.5412445730824892
Clinical Algorithm: 0.6069102749638206
Pharmacogenetic Algorithm: 0.6617221418234442


# Build an oracle for the multi-armed bandit setting
Following the Lasso Bandit paper, we establish an approximate oracle that estimates the true parameters of each arm using all of the data. Basically, we train a Logistic classifier for each arm (low, medium, high) using all of the data with no regularization.

In [5]:
X = data.drop(['daily-dosage', 'dosage-level'], axis=1)
y = [(data['dosage-level'] == level).astype(np.float32).values for level in ('low', 'medium', 'high')]

# Train three Logistic Classifiers for the three arms
models = [LogisticRegression(C=100000, solver='liblinear') for _ in range(3)]  # large C -> no regularization
for i in range(3):
    models[i].fit(X.values, y[i])

prediction_score = np.array([m.predict_proba(X)[:,1] for m in models])
prediction_class = np.argmax(prediction_score, axis=0)  # Choose the arm with highest score
correct_class = data['dosage-level'].map({'low': 0, 'medium': 1, 'high': 2}).values
print("oracle accuracy: {}".format(accuracy(correct_class, prediction_class)))

oracle accuracy: 0.6863241678726484


Quick sanity check: Train linear regression on this dataset with no regularization. This should give better performance compared to the baseline algorithm, although the resulting model might not generalize well.

In [6]:
model = LinearRegression()
model.fit(X, np.sqrt(data['daily-dosage'].values))

# Evaluate performance
print(accuracy(data['dosage-level'].values, pd.Series(model.predict(X)**2).map(discretize_label).values))

0.6814399421128798
